# Enabling ML explainability with SageMaker Clarify

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
%store -r s3_bucket_name
%store -r prefix
%store -r training_data_path
%store -r test_data_path
%store -r model_name

In [ ]:
import sagemaker

session = sagemaker.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()

In [ ]:
s3_training_data_path = training_data_path
s3_test_data_path = test_data_path
s3_output_path = f"s3://{s3_bucket_name}/{prefix}/output"

In [ ]:
!aws s3 cp {s3_training_data_path} tmp/training_data.csv
!aws s3 cp {s3_test_data_path} tmp/test_data.csv

In [ ]:
import pandas as pd

training_data = pd.read_csv("tmp/training_data.csv")
test_data = pd.read_csv("tmp/test_data.csv")

target = test_data['approved']
features = test_data.drop(columns=['approved'])
features.to_csv('tmp/test_features.csv', index=False, header=False)

In [ ]:
features

In [ ]:
base = f"s3://{s3_bucket_name}/{prefix}/input"
s3_feature_path = f"{base}/test_features.csv"

!aws s3 cp tmp/test_features.csv {s3_feature_path}

In [ ]:
from sagemaker.clarify import ModelConfig

model_config = ModelConfig(
    model_name=model_name,
    instance_type='ml.c5.xlarge',
    instance_count=1,
    accept_type='text/csv'
)

In [ ]:
from sagemaker.clarify import SageMakerClarifyProcessor

processor = SageMakerClarifyProcessor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    sagemaker_session=session
)

In [ ]:
baseline = features.iloc[0:200].values.tolist()
baseline

In [ ]:
from sagemaker.clarify import SHAPConfig

shap_config = SHAPConfig(
    baseline=baseline,
    num_samples=50,
    agg_method='median'
)

In [ ]:
headers = training_data.columns.to_list()

In [ ]:
from sagemaker.clarify import DataConfig

data_config = DataConfig(
    s3_data_input_path=s3_training_data_path,
    s3_output_path=s3_output_path,
    label='approved',
    headers=headers,
    dataset_type='text/csv'
)

In [ ]:
%%time

processor.run_explainability(
    data_config=data_config,       
    model_config=model_config,                                 
    explainability_config=shap_config
)

In [ ]:
output = processor.latest_job.outputs[0]
output_destination = output.destination
output_destination

In [ ]:
!aws s3 cp {output_destination}/ tmp/ --recursive

In [ ]:
!ls -lahF tmp/

In [ ]:
!cat tmp/analysis.json